# Using UMLS Concepts with MeSH

The Medical Subject Headings (MeSH) terms returned from a PubMed search can be further analyzed
by mapping them to Unified Medical Language System (UMLS) concepts, as well as
filtering the MeSH Terms by concepts.

For both mapping MeSH to UMLS Concepts and filtering MeSH by concept, the following backends are supported:
* MySQL
* SQLite
* DataFrames

### Set Up

In [1]:
using SQLite
using MySQL
using BioMedQuery.DBUtils
using BioMedQuery.Processes
using BioServices.UMLS
using BioMedQuery.PubMed
using DataFrames

Credentials are environment variables (e.g set in your .juliarc.jl)

In [2]:
umls_user = ENV["UMLS_USER"];
umls_pswd = ENV["UMLS_PSSWD"];
email = ""; # Only needed if you want to contact NCBI with inqueries
search_term = """(obesity[MeSH Major Topic]) AND ("2010"[Date - Publication] : "2012"[Date - Publication])""";
umls_concept = "Disease or Syndrome";
max_articles = 5;
results_dir = ".";
verbose = true;

results_dir = ".";

## MySQL

### Map Medical Subject Headings (MeSH) to UMLS

This example demonstrates the typical workflow to populate a MESH2UMLS database
table relating all concepts associated with all MeSH terms in the input database.

*Note: this example reuses the MySQL DB from the PubMed Search and Save example.*

Create MySQL DB connection

In [3]:
host = "127.0.0.1";
mysql_usr = "root";
mysql_pswd = "";
dbname = "pubmed_obesity_2010_2012";

db_mysql = MySQL.connect(host, mysql_usr, mysql_pswd, db = dbname);

Map MeSH to UMLS

In [4]:
@time map_mesh_to_umls_async!(db_mysql, umls_user, umls_pswd; append_results=false, timeout=3);

  likely near string:1
Add `using Dates` to your imports.
  likely near string:1
  likely near string:1
  likely near string:1
Add `using Dates` to your imports.
  likely near string:1
  likely near string:1
----------Matching MESH to UMLS-----------
["Adult", "Aged", "Aged, 80 and over", "Analysis of Variance", "Body Weight", "C-Reactive Protein", "Child", "Cross-Sectional Studies", "Fatigue", "Female", "Fibromyalgia", "Germany", "Health Status", "Humans", "Japan", "Male", "Middle Aged", "Nutrition Surveys", "Obesity", "Pain", "Pain Measurement", "Physical Fitness", "Prognosis", "Quality of Life", "Surveys and Questionnaires", "Reference Values", "Risk Factors", "ROC Curve", "Severity of Illness Index", "Sports", "Television", "Thyrotropin", "Biomarkers", "Weight Gain", "Exercise", "Body Mass Index", "Incidence", "Prevalence", "Logistic Models", "Odds Ratio", "Case-Control Studies", "Age Distribution", "Sex Distribution", "Sleep Apnea, Obstructive", "Metabolic Syndrome", "Overweight",

#### Explore the output table

In [5]:
db_query(db_mysql, "SELECT * FROM mesh2umls")

,mesh,umls
1,Adult,Age Group
2,Age Distribution,Quantitative Concept
3,Aged,Organism Attribute
4,"Aged, 80 and over",Age Group
5,Analysis of Variance,Quantitative Concept
6,Biomarkers,Clinical Attribute
7,Body Mass Index,Diagnostic Procedure
8,Body Weight,Organism Attribute
9,C-Reactive Protein,"Amino Acid, Peptide, or Protein"
10,C-Reactive Protein,Immunologic Factor


### Filtering MeSH terms by UMLS concept

Getting the descriptor to index dictionary and the occurence matrix

In [6]:
@time labels2ind, occur = umls_semantic_occurrences(db_mysql, umls_concept);

Filter mesh query string : SELECT mesh FROM mesh2umls WHERE umls IN ('Disease or Syndrome')
-------------------------------------------------------------
Found 5 articles with valid descriptors
-------------------------------------------------------------
  1.185141 seconds (1.80 M allocations: 91.534 MiB, 4.03% gc time)


Descriptor to Index Dictionary

In [7]:
labels2ind

Dict{String,Int64} with 5 entries:
  "Obesity"                  => 1
  "Pediatric Obesity"        => 2
  "Sleep Apnea, Obstructive" => 3
  "Metabolic Syndrome"       => 4
  "Fibromyalgia"             => 5

Output Data Matrix

In [8]:
full(occur)

┌ Warning: `full(S::SparseMatrixCSC)` (and `full` in general) has been deprecated. To replace `full(S::SparseMatrixCSC)`, consider `Matrix(S)` or, if that option is too narrow, `Array(S)`.
│   caller = top-level scope at none:0
└ @ Core none:0


5×5 Array{Float64,2}:
 1.0  1.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0

## SQLite

This example demonstrates the typical workflow to populate a MESH2UMLS database
table relating all concepts associated with all MeSH terms in the input database.

*Note: this example reuses the SQLite DB from the PubMed Search and Save example.*

Create SQLite DB connection

In [9]:
db_path = "$(results_dir)/pubmed_obesity_2010_2012.db";
db_sqlite = SQLite.DB(db_path);

if isfile(db_path) # hide
    rm(db_path) # hide
end # hide
db_sqlite = SQLite.DB(db_path); # hide
PubMed.create_tables!(db_sqlite); # hide
Processes.pubmed_search_and_save!(email, search_term, max_articles, db_sqlite, false) # hide

Getting 5 articles, starting at index 0
------ESearch--------
------EFetch--------
------Save to database--------
Saving 5 articles to database
Finished searching, total number of articles: 5


### Map MeSH to UMLS

In [10]:
@time map_mesh_to_umls_async!(db_sqlite, umls_user, umls_pswd; append_results=false, timeout=3);

----------Matching MESH to UMLS-----------
Union{Missing, String}["Reference Values", "Republic of Korea", "ROC Curve", "Fatigue", "Obesity", "Risk Factors", "Logistic Models", "Severity of Illness Index", "Male", "Case-Control Studies", "Analysis of Variance", "Sedentary Lifestyle", "Prevalence", "Quality of Life", "Odds Ratio", "Exercise", "Body Mass Index", "Aged", "Child", "Sex Distribution", "Adult", "Germany", "Sports", "Thyrotropin", "Pediatric Obesity", "Humans", "Japan", "Cross-Sectional Studies", "Weight Gain", "Middle Aged", "Surveys and Questionnaires", "Health Status", "Young Adult", "Incidence", "Prognosis", "Body Weight", "Pain Measurement", "Waist Circumference", "Metabolic Syndrome", "Pain", "Nutrition Surveys", "Fibromyalgia", "Sleep Apnea, Obstructive", "Television", "Age Distribution", "Overweight", "Physical Fitness", "Female", "Biomarkers", "Obesity, Abdominal", "C-Reactive Protein", "Aged, 80 and over"]
┌ Warning: `info()` is deprecated, use `@info` instead.
│   

Explore the output table

In [11]:
db_query(db_sqlite, "SELECT * FROM mesh2umls;")

,mesh,umls
1,Republic of Korea,Geographic Area
2,Severity of Illness Index,Quantitative Concept
3,Case-Control Studies,Research Activity
4,Obesity,Disease or Syndrome
5,Thyrotropin,"Amino Acid, Peptide, or Protein"
6,Thyrotropin,Hormone
7,Thyrotropin,Pharmacologic Substance
8,Risk Factors,Finding
9,Weight Gain,Finding
10,Body Mass Index,Diagnostic Procedure


### Filtering MeSH terms by UMLS concept

Getting the descriptor to index dictionary and occurence matrix

In [12]:
@time labels2ind, occur = umls_semantic_occurrences(db_sqlite, umls_concept);

Filter mesh query string : SELECT mesh FROM mesh2umls WHERE umls IN ('Disease or Syndrome')
-------------------------------------------------------------
Found 5 articles with valid descriptors
-------------------------------------------------------------
  0.576257 seconds (790.14 k allocations: 40.460 MiB, 4.86% gc time)


Descriptor to Index Dictionary

In [13]:
labels2ind

Dict{String,Int64} with 5 entries:
  "Obesity"                  => 1
  "Sleep Apnea, Obstructive" => 2
  "Pediatric Obesity"        => 3
  "Metabolic Syndrome"       => 4
  "Fibromyalgia"             => 5

Output Data Matrix

In [14]:
Matrix(occur)

5×5 Array{Float64,2}:
 1.0  1.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0

## DataFrames

This example demonstrates the typical workflow to create a MeSH to UMLS map as a DataFrame
relating all concepts associated with all MeSH terms in the input dataframe.

Get the articles (same as example in PubMed Search and Parse)

In [15]:
dfs = Processes.pubmed_search_and_parse(email, search_term, max_articles, verbose)

Getting 5 articles, starting at index 0
------ESearch--------
------EFetch--------
------Save to dataframes--------


Dict{String,DataFrames.DataFrame} with 8 entries:
  "basic"               => 5×13 DataFrames.DataFrame. Omitted printing of 9 col…
  "mesh_desc"           => 52×2 DataFrames.DataFrame…
  "mesh_qual"           => 9×2 DataFrames.DataFrame…
  "pub_type"            => 10×3 DataFrames.DataFrame…
  "abstract_full"       => 5×2 DataFrames.DataFrame. Omitted printing of 1 colu…
  "author_ref"          => 35×8 DataFrames.DataFrame. Omitted printing of 3 col…
  "mesh_heading"        => 78×5 DataFrames.DataFrame…
  "abstract_structured" => 4×4 DataFrames.DataFrame. Omitted printing of 1 colu…

Map MeSH to UMLS and explore the output table

In [16]:
@time res = map_mesh_to_umls_async(dfs["mesh_desc"], umls_user, umls_pswd)

----------Matching MESH to UMLS-----------
Any["Reference Values", "Republic of Korea", "ROC Curve", "Fatigue", "Obesity", "Risk Factors", "Logistic Models", "Severity of Illness Index", "Male", "Case-Control Studies", "Analysis of Variance", "Sedentary Lifestyle", "Prevalence", "Quality of Life", "Odds Ratio", "Exercise", "Body Mass Index", "Aged", "Child", "Sex Distribution", "Adult", "Germany", "Sports", "Thyrotropin", "Pediatric Obesity", "Humans", "Japan", "Cross-Sectional Studies", "Weight Gain", "Middle Aged", "Surveys and Questionnaires", "Health Status", "Young Adult", "Incidence", "Prognosis", "Body Weight", "Pain Measurement", "Waist Circumference", "Metabolic Syndrome", "Pain", "Nutrition Surveys", "Fibromyalgia", "Sleep Apnea, Obstructive", "Television", "Age Distribution", "Overweight", "Physical Fitness", "Female", "Biomarkers", "Obesity, Abdominal", "C-Reactive Protein", "Aged, 80 and over"]
┌ Warning: `info()` is deprecated, use `@info` instead.
│   caller = #get_tgt#2

,descriptor,concept
1,Adult,Age Group
2,Age Distribution,Quantitative Concept
3,Aged,Organism Attribute
4,"Aged, 80 and over",Age Group
5,Analysis of Variance,Quantitative Concept
6,Biomarkers,Clinical Attribute
7,Body Mass Index,Diagnostic Procedure
8,Body Weight,Organism Attribute
9,C-Reactive Protein,"Amino Acid, Peptide, or Protein"
10,C-Reactive Protein,Immunologic Factor


Getting the descriptor to index dictionary and occurence matrix

In [17]:
@time labels2ind, occur = umls_semantic_occurrences(dfs, res, umls_concept);

-------------------------------------------------------------
Found 5 articles with valid descriptors
-------------------------------------------------------------
  0.594624 seconds (890.71 k allocations: 44.498 MiB, 2.42% gc time)


Descriptor to Index Dictionary

In [18]:
labels2ind

Dict{String,Int64} with 5 entries:
  "Obesity"                  => 1
  "Pediatric Obesity"        => 2
  "Sleep Apnea, Obstructive" => 3
  "Metabolic Syndrome"       => 4
  "Fibromyalgia"             => 5

Output Data Matrix

In [19]:
Matrix(occur)

5×5 Array{Float64,2}:
 0.0  1.0  0.0  1.0  1.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*